## 패키지 선언

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader


## Dataset 선언

In [ ]:
# Training dataset 다운로드
cifar10_train = dataset.CIFAR10(root = "./", # 데이터셋을 저장할 위치
                            train = True,
                            transform = transform.ToTensor(),
                            download = True)
# Testing dataset 다운로드
cifar10_test = dataset.CIFAR10(root = "./",
                            train = False,
                            transform = transform.ToTensor(),
                            download = True)

## CIFAR10 데이터셋 형상 확인

In [ ]:
from matplotlib import pyplot as plt
print(len(cifar10_train))     # training dataset 개수 확인

first_data = cifar10_train[1]
print(first_data[0].shape)  # 두번째 data의 형상 확인
print(first_data[1])        # 두번째 data의 정답 확인


plt.imshow(first_data[0].permute(1, 2, 0))
plt.show()

## VGG Net 모델 정의

In [ ]:
class VGG (nn.Module):
  def __init__ (self):
    super(VGG, self).__init__()

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)    # Convolution: [3x3x3] x 16, s1, p1
    self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x16] x 32, s1, p1

    self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x32] x 32, s1, p1
    self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   # Convolution: [3x3x64] x 64, s1, p1

    self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # Convolution: [3x3x64] x 128, s1, p1
    self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # Convolution: [3x3x128] x 256, s1, p1

    self.fc1 = nn.Linear(4096, 512)   # Fully connected layer: 4096 x 512
    self.fc2 = nn.Linear(512, 256)    # Fully connected layer: 512 x 256
    self.fc3 = nn.Linear(256, 10)     # Fully connected layer: 256 x 10

    # 파라미터를 가지지 않은 layer는 한 번만 선언해도 문제 없음
    self.relu = nn.ReLU()
    self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)


  def forward(self, x):

    # convolution layers
    out = self.relu(self.conv1_1(x))
    out = self.relu(self.conv1_2(out))
    out = self.avgPool2d(out)

    out = self.relu(self.conv2_1(out))
    out = self.relu(self.conv2_2(out))
    out = self.avgPool2d(out)

    out = self.relu(self.conv3_1(out))
    out = self.relu(self.conv3_2(out))
    out = self.avgPool2d(out)

    # 평탄화
    out = out.reshape(-1, 4096)

    # fully connected layers
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    out = self.fc3(out)

    return out

## Hyper-parameters 지정

In [ ]:
batch_size = 100
learning_rate = 0.1
training_epochs = 20
loss_function = nn.CrossEntropyLoss()
network = VGG()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)
data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

## 학습을 위한 반복문 선언

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

network = network.to(device)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:

    img = img.to(device)
    label = label.to(device)

    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

## 학습이 완료된 모델을 이용해 정답률 확인

In [ ]:
network = network.to('cpu')
with torch.no_grad(): # test에서는 기울기 계산 제외

  img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255.
  label_test = torch.tensor(cifar10_test.targets)

  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

## -----------------------------------------------------------------

## Model architecture 구성 실습 (ResNet)

## 모델 정의

In [ ]:
class ResNet (nn.Module):
  def __init__ (self):
    super(ResNet, self).__init__()

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)    # Convolution: [3x3x3] x 16, s1, p1
    self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x16] x 32, s1, p1

    self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x32] x 32, s1, p1
    self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   # Convolution: [3x3x64] x 64, s1, p1

    self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # Convolution: [3x3x64] x 128, s1, p1
    self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # Convolution: [3x3x128] x 256, s1, p1

    self.fc1 = nn.Linear(4096, 512)   # Fully connected layer: 4096 x 512
    self.fc2 = nn.Linear(512, 256)    # Fully connected layer: 512 x 256
    self.fc3 = nn.Linear(256, 10)     # Fully connected layer: 256 x 10

    # 파라미터를 가지지 않은 layer는 한 번만 선언해도 문제 없음
    self.relu = nn.ReLU()
    self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)

    # skip connection
    self.conv_skip1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.conv_skip2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.conv_skip3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)

  def forward(self, x):

    # convolution layers
    input_feature = x
    out = self.relu(self.conv1_1(x))
    out = self.relu(self.conv1_2(out))

    skip = self.conv_skip1(input_feature)
    out = out + skip

    out = self.avgPool2d(out)

    input_feature = out

    out = self.relu(self.conv2_1(out))
    out = self.relu(self.conv2_2(out))

    skip = self.conv_skip2(input_feature)
    out = out + skip

    out = self.avgPool2d(out)

    out = self.relu(self.conv3_1(out))
    out = self.relu(self.conv3_2(out))
    out = self.avgPool2d(out)

    # 평탄화
    out = out.reshape(-1, 4096)

    # fully connected layers
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    out = self.fc3(out)

    return out

## Training

In [ ]:
# Hyper-parameters 지정
batch_size = 100
learning_rate = 0.1
training_epochs = 20
loss_function = nn.CrossEntropyLoss()
network = ResNet()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)
data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

# 학습을 위한 반복문 진행
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

network = network.to(device)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:

    img = img.to(device)
    label = label.to(device)

    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

# 정답률 확인
network = network.to('cpu')
with torch.no_grad(): # test에서는 기울기 계산 제외

  img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255.
  label_test = torch.tensor(cifar10_test.targets)

  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

## -----------------------------------------------------------------

## Model architecture 구성 실습 (DenseNet)

## 모델 정의

In [ ]:
class DenseNet(nn.Module):
  def __init__ (self):
    super(DenseNet, self).__init__()

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)    # Convolution: [3x3x3] x 16, s1, p1
    self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x16] x 32, s1, p1

    self.conv2_1 = nn.Conv2d(in_channels=35, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x32] x 32, s1, p1
    self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   # Convolution: [3x3x64] x 64, s1, p1

    self.conv3_1 = nn.Conv2d(in_channels=99, out_channels=128, kernel_size=3, padding=1)  # Convolution: [3x3x64] x 128, s1, p1
    self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # Convolution: [3x3x128] x 256, s1, p1

    self.fc1 = nn.Linear(4096, 512)   # Fully connected layer: 4096 x 512
    self.fc2 = nn.Linear(512, 256)    # Fully connected layer: 512 x 256
    self.fc3 = nn.Linear(256, 10)     # Fully connected layer: 256 x 10

    # 파라미터를 가지지 않은 layer는 한 번만 선언해도 문제 없음
    self.relu = nn.ReLU()
    self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)


  def forward(self, x):

    # convolution layers
    input_feature = x
    out = self.relu(self.conv1_1(x))
    out = self.relu(self.conv1_2(out))

    out = torch.cat([out, input_feature], dim=1)
    out = self.avgPool2d(out)

    input_feature = out

    out = self.relu(self.conv2_1(out))
    out = self.relu(self.conv2_2(out))

    out = torch.cat([out, input_feature], dim=1)
    out = self.avgPool2d(out)

    out = self.relu(self.conv3_1(out))
    out = self.relu(self.conv3_2(out))
    out = self.avgPool2d(out)

    # 평탄화
    out = out.reshape(-1, 4096)

    # fully connected layers
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    out = self.fc3(out)

    return out

## Training

In [ ]:
# Hyper-parameters 지정
batch_size = 100
learning_rate = 0.1
training_epochs = 20
loss_function = nn.CrossEntropyLoss()
network = DenseNet()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)
data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

# 학습을 위한 반복문 진행
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

network = network.to(device)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:

    img = img.to(device)
    label = label.to(device)

    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

# 정답률 확인
network = network.to('cpu')
with torch.no_grad(): # test에서는 기울기 계산 제외

  img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255.
  label_test = torch.tensor(cifar10_test.targets)

  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

## -----------------------------------------------------------------:

## Model architecture 구성 실습 (SENet)

## 모델 정의

In [ ]:
class SENet (nn.Module):
  def __init__ (self):
    super(SENet, self).__init__()

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)    # Convolution: [3x3x3] x 16, s1, p1
    self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x16] x 32, s1, p1

    self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x32] x 32, s1, p1
    self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   # Convolution: [3x3x64] x 64, s1, p1

    self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # Convolution: [3x3x64] x 128, s1, p1
    self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # Convolution: [3x3x128] x 256, s1, p1

    self.fc1 = nn.Linear(4096, 512)   # Fully connected layer: 4096 x 512
    self.fc2 = nn.Linear(512, 256)    # Fully connected layer: 512 x 256
    self.fc3 = nn.Linear(256, 10)     # Fully connected layer: 256 x 10

    # 파라미터를 가지지 않은 layer는 한 번만 선언해도 문제 없음
    self.relu = nn.ReLU()
    self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)

    # channel attention
    self.adaptivePool = nn.AdaptiveAvgPool2d((1, 1))
    self.caconv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
    self.caconv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
    self.sigmoid = nn.Sigmoid()




  def forward(self, x):

    # convolution layers
    out = self.relu(self.conv1_1(x))
    out = self.relu(self.conv1_2(out))
    out = self.avgPool2d(out)

    out = self.relu(self.conv2_1(out))
    out = self.relu(self.conv2_2(out))
    out = self.avgPool2d(out)

    ca_out = self.adaptivePool(out)
    ca_out = self.caconv1(ca_out)
    ca_out = self.relu(ca_out)
    ca_out = self.caconv2(ca_out)
    ca_out = self.sigmoid(ca_out)
    ca_out = ca_out.expand_as(out)
    out = out * ca_out

    out = self.relu(self.conv3_1(out))
    out = self.relu(self.conv3_2(out))
    out = self.avgPool2d(out)

    # 평탄화
    out = out.reshape(-1, 4096)

    # fully connected layers
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    out = self.fc3(out)

    return out

## Training

In [ ]:
# Hyper-parameters 지정
batch_size = 100
learning_rate = 0.1
training_epochs = 20
loss_function = nn.CrossEntropyLoss()
network = SENet()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)
data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

# 학습을 위한 반복문 진행
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

network = network.to(device)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:

    img = img.to(device)
    label = label.to(device)

    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

# 정답률 확인
network = network.to('cpu')
with torch.no_grad(): # test에서는 기울기 계산 제외

  img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255.
  label_test = torch.tensor(cifar10_test.targets)

  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

## -----------------------------------------------------------------


## Model architecture 구성 실습 (조합 실험)

In [ ]:
class CombiNet (nn.Module):
  def __init__ (self):
    super(CombiNet, self).__init__()

    self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)    # Convolution: [3x3x3] x 16, s1, p1
    self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x16] x 32, s1, p1

    self.conv2_1 = nn.Conv2d(in_channels=35, out_channels=32, kernel_size=3, padding=1)   # Convolution: [3x3x32] x 32, s1, p1
    self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)   # Convolution: [3x3x64] x 64, s1, p1

    self.conv3_1 = nn.Conv2d(in_channels=99, out_channels=128, kernel_size=3, padding=1)  # Convolution: [3x3x64] x 128, s1, p1
    self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # Convolution: [3x3x128] x 256, s1, p1

    self.fc1 = nn.Linear(4096, 512)   # Fully connected layer: 4096 x 512
    self.fc2 = nn.Linear(512, 256)    # Fully connected layer: 512 x 256
    self.fc3 = nn.Linear(256, 10)     # Fully connected layer: 256 x 10

    # 파라미터를 가지지 않은 layer는 한 번만 선언해도 문제 없음
    self.relu = nn.ReLU()
    self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)
    self.maxPool2d = nn.MaxPool2d(kernel_size=2, stride=2)

    # Skip conv
    self.conv_skip1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.conv_skip2 = nn.Conv2d(in_channels=35, out_channels=64, kernel_size=3, padding=1)
    self.conv_skip3 = nn.Conv2d(in_channels=99, out_channels=256, kernel_size=3, padding=1)

    # channel attention
    self.adaptivePool = nn.AdaptiveAvgPool2d((1, 1))
    self.caconv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
    self.caconv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
    self.sigmoid = nn.Sigmoid()




  def forward(self, x):
    # Layer 1
    # convolution layers
    temp_input = x
    out = self.relu(self.conv1_1(x))
    out = self.relu(self.conv1_2(out))

    # Skip connection
    skip = self.relu(self.conv_skip1(temp_input))
    out = out + skip

    # Dense connection
    out = torch.cat([out, temp_input], dim=1)

    # Layer 2
    # Convolution layers
    out = self.maxPool2d(out)
    temp_input = out
    out = self.relu(self.conv2_1(out))
    out = self.relu(self.conv2_2(out))

    # Channel attention
    ca_out = self.adaptivePool(out)
    ca_out = self.caconv1(ca_out)
    ca_out = self.relu(ca_out)
    ca_out = self.caconv2(ca_out)
    ca_out = self.sigmoid(ca_out)
    ca_out = ca_out.expand_as(out)
    out = out * ca_out

    # Skip connection
    skip = self.relu(self.conv_skip2(temp_input))
    out = out + skip

    # Dense connection
    out = torch.cat([out, temp_input], dim=1)

    # Layer 3
    # Convolution layers
    out = self.maxPool2d(out)
    temp_input = out
    out = self.relu(self.conv3_1(out))
    out = self.relu(self.conv3_2(out))

    # Skip connection
    skip = self.relu(self.conv_skip3(temp_input))
    out = out + skip
    out = self.maxPool2d(out)

    # 평탄화
    out = out.reshape(-1, 4096)

    # fully connected layers
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    out = self.fc3(out)

    return out

## Training

In [ ]:
# Hyper-parameters 지정
batch_size = 100
learning_rate = 0.1
training_epochs = 20
loss_function = nn.CrossEntropyLoss()
network = CombiNet()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)
data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

# 학습을 위한 반복문 진행
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

network = network.to(device)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:

    img = img.to(device)
    label = label.to(device)

    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

# 정답률 확인
network = network.to('cpu')
with torch.no_grad(): # test에서는 기울기 계산 제외

  img_test = torch.tensor(np.transpose(cifar10_test.data, (0, 3, 1, 2)))/255.
  label_test = torch.tensor(cifar10_test.targets)

  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())